## Create CONSTANTS

In [28]:
DATASET = [
    {
        'Tid': 'T1',
        'Item': ['a', 'b', 'c', 'd'],
        'Quantity': [5,2,1,2]
    },
    {
        'Tid': 'T2',
        'Item': ['a', 'c', 'd', 'g'],
        'Quantity': [1,1,1,3]
    },
    {
        'Tid': 'T3',
        'Item': ['a', 'c', 'f'],
        'Quantity': [1,1,1]
    },
    {
        'Tid': 'T4',
        'Item': ['a', 'f', 'g'],
        'Quantity': [1,4,2]
    },
    {
        'Tid': 'T5',
        'Item': ['a', 'g'],
        'Quantity': [1,2]
    },
    {
        'Tid': 'T6',
        'Item': ['b', 'c', 'd', 'e'],
        'Quantity': [3,2,3,1]
    },
    {
        'Tid': 'T7',
        'Item': ['c', 'e'],
        'Quantity': [6,4]
    },
    {
        'Tid': 'T8',
        'Item': ['e', 'f'],
        'Quantity': [1,3]
    }
]
UNIT_UTILITY = {
    'a': 3,
    'b': 6,
    'c': -3,
    'd': 12,
    'e': -5,
    'f': -2,
    'g': -1
}

EPM = {
    'min_util': 30,
    'min_per': 1,
    'max_per': 5,
    'min_avg': 1,
    'max_avg': 3
}

### Show global variable

In [31]:
print(DATASET[0])
print(UNIT_UTILITY)

{'Tid': 'T1', 'Item': ['a', 'b', 'c', 'd'], 'Quantity': [5, 2, 1, 2]}
{'a': 3, 'b': 6, 'c': -3, 'd': 12, 'e': -5, 'f': -2, 'g': -1}


In [32]:
print(EPM)

{'min_util': 30, 'min_per': 1, 'max_per': 5, 'min_avg': 1, 'max_avg': 3}


## Preparation Procedure

The Transaction Weight Utility (TWU) of item \( i \) is given by:

Def 6
Def 4
\[
{TWU}(i) = \sum_{{X \in T_j \\ T_j \subseteq D}} P_U(T_j)
\]

calculate maxPer(i), avgPer(i)
Def 1, Def 2

rearrange order
Def 7


In [37]:
def create_periodic_high_util_itemset(dataset: list, unit_utility: dict, epm: dict) -> list:
    # calculate TWU of each item in dataset
    twu_dict = dict()
    ps_dict = dict()
    transaction_dict = dict()
    for i in dataset:
        for j in range(len(i['Item'])):
            if i['Item'][j] not in twu_dict:
                twu_dict[i['Item'][j]] = 0
                transaction_dict[i['Item'][j]] = []
                ps_dict[i['Item'][j]] = []
            twu_dict[i['Item'][j]] += i['Quantity'][j] * unit_utility[i['Item'][j]] if unit_utility[i['Item'][j]] > 0 else 0
            transaction_dict[i['Item'][j]].append(int(i['Tid'][1:]))
    
    for i in transaction_dict.keys():
        for j in range(1, len(transaction_dict[i])):
            ps_dict[i].append(transaction_dict[i][j] - transaction_dict[i][j - 1])
    
    temp_dataset = dataset

    for i in temp_dataset:
        temp_item_quantity = list()
        for j in range(len(i['Item'])):
            avg_item = sum(ps_dict[i['Item'][j]])/len(ps_dict[i['Item'][j]])
            if twu_dict[i['Item'][j]] > epm['min_util'] and max(ps_dict[i['Item'][j]]) <= epm['max_per']\
            and avg_item >= epm['min_avg'] and avg_item <= epm['max_avg']:
                
                temp_item_quantity.append((i['Item'][j], i['Quantity'][j], unit_utility[i['Item'][j]], twu_dict[i['Item'][j]]))

        temp_item_quantity.sort(key=lambda x: x[3])
        temp_item_quantity.sort(key=lambda x: x[2])
        i['Item'] = list(map(temp_item_quantity, lambda x: x[0]))
        i['Quantity'] = list(map(temp_item_quantity, lambda x: x[1]))



In [38]:
create_periodic_high_util_itemset(dataset=DATASET, unit_utility=UNIT_UTILITY, epm=EPM)

27
30
0
72
0
0
0
[1, 2, 3, 4, 5]
[1, 6]
[1, 2, 3, 6, 7]
[1, 2, 6]
[2, 4, 5]
[3, 4, 8]
[6, 7, 8]
